In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cPickle

import analysis
reload(analysis)

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


<module 'analysis' from 'analysis.pyc'>

In [21]:
fname = '/home/jesse/Downloads/comparison_128_2.pkl'

A_priors = ['L2', 'L4', 'RANDOM']
models = [2, 4, 6, 8, 'RANDOM', 'COULOMB', 'COHERENCE'] + ['SC']
lambdas = np.logspace(-2, 2, num=9)
colors = ['black', 'black', 'black', 'black', 'red', 'green', 'blue', 'purple']
fmts = ['-', '--', ':', '-.', '-', '-', '-', '-']

In [3]:
with open(fname, 'r') as f:
    data = cPickle.load(f)

In [22]:
A_dict, W_dict, W_fits, min_ks, results = data
n_mixtures, n_sources = A_dict[A_dict.keys()[0]][0].shape
n_iter = W_fits.shape[3]
results = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, 3))
max_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources))
other_angles = np.nan * np.ones((len(A_priors), len(models), lambdas.size, n_iter, n_sources**2-n_sources))

In [23]:
for ii, p in enumerate(A_priors):
    for jj, m in enumerate(models):
        A = A_dict[p][jj]
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                W = W_fits[ii, jj, kk, ll]
                hd, (pd, ma, oa, mod) = analysis.recovery_statistics_AW(A, W)
                results[ii, jj, kk, ll] = [hd, pd, mod]
                max_angles[ii, jj, kk, ll] = ma
                other_angles[ii, jj, kk, ll] = oa

In [24]:
f, axes = plt.subplots(len(A_priors), 4, figsize=(12, 12))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    (ad, ae, ap, aa) = a
    for jj, m in enumerate(models):
        if ii == 0:
            label = str(m)
            ad.legend(loc='best', prop={'size': 12})
        else:
            label = None
        ma = np.median(max_angles[ii, jj, :, :], axis=-1)
        oa = np.median(other_angles[ii, jj, :, :], axis=-1)
        da = (ma - oa).mean(axis=1)
        dist = results[ii, jj, :, :, 0].mean(axis=1)
        delta = results[ii, jj, :, :, 1].mean(axis=1)
        distp = results[ii, jj, :, :, 2].mean(axis=1)
        ad.semilogx(lambdas, dist, fmts[jj], label=label, c=colors[jj])
        ad.semilogx(lambdas, dist, '.', c=colors[jj])
        ae.semilogx(lambdas, delta, fmts[jj], label=label, c=colors[jj])
        ae.semilogx(lambdas, delta, '.', c=colors[jj])
        ap.semilogx(lambdas, distp, fmts[jj], label=label, c=colors[jj])
        ap.semilogx(lambdas, distp, '.', c=colors[jj])
        aa.semilogx(lambdas, da, fmts[jj], label=label, c=colors[jj])
        aa.semilogx(lambdas, da, '.', c=colors[jj])

    ad.set_title(p)
    ad.set_xlabel('Lambda')
    ad.set_ylabel('Hellinger Distance')
    ae.set_title(p)
    ae.set_xlabel('Lambda')
    ae.set_ylabel('Permutation Delta')
    ap.set_title(p)
    ap.set_xlabel('Lambda')
    ap.set_ylabel('Hellinger Distance Max vs. Other')
    aa.set_title(p)
    aa.set_xlabel('Lambda')
    aa.set_ylabel('Difference in Median Max vs. Other')
f.tight_layout()
#    plt.savefig(p+'.pdf')
plt.show()
plt.close()

In [ ]:
f, axes = plt.subplots(len(A_priors), len(models), figsize=(24, 8))
for ii, (p, a) in enumerate(zip(A_priors, axes)):
    for jj, (m, ax) in enumerate(zip(models, a)):
        ma = max_angles[ii, jj, 4, :]
        oa = other_angles[ii, jj, 4, :]
        md, bins = analysis.angle_histogram(ma)
        od, bins = analysis.angle_histogram(oa)
        ax.semilogy(bins[:-1], md, drawstyle='steps', color='yellow', lw=1.5)
        ax.semilogy(bins[:-1], od, drawstyle='steps', color='green', lw=1.5)

        ax.set_title(p)
        ax.set_xlabel('Angle')
        ax.set_ylabel(str(m))
        ax.set_xticks([0, 45, 90])
        ax.set_yticks([1e-6, 1e-3, 1e0])
f.tight_layout()
#    plt.savefig(p+'.pdf')
plt.show()
plt.close()

In [5]:
import h5py
import analysis

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


In [3]:
with h5py.File('/home/jesse/Downloads/a_array-128_OC-2.0_priors-L2_L4_RANDOM_COHERENCE.h5') as f:
    A_array = f['A_array'].value
    A_priors = f['A_priors'].value

In [11]:
for ii, p in enumerate(A_priors):
    print p, analysis.find_max_allowed_k(A_array[ii])

L2 2
L4 6
RANDOM 4
COHERENCE 5
